In [62]:
!pip install pandas psycopg2-binary matplotlib plotly

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 13.2 MB 3.2 MB/s eta 0:00:01
  Using cached retrying-1.3.3.tar.gz (10 kB)
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11430 sha256=b28902db42fe9a5b4b4e67c0003ec4b48ccb6c91c4259d38fabd6e480339defb
  Stored in directory: /home/gabriel/.cache/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import psycopg2 as pg

connection = pg.connect("host=localhost dbname=postgres user=postgres password=postgres")

In [10]:
student_count_per_city_9th_grade_query = """
select
	city_name as city,
	count(1) as students_count
from
	public.censo
where
	enrollment_stage_name = 'Ensino Fundamental de 9 anos - 9º Ano'
group by
	city_name 
order by
	students_count DESC
limit 10
"""

In [11]:
student_count_per_city_9th_grade = pd.read_sql_query(student_count_per_city_9th_grade_query, connection)
student_count_per_city_9th_grade

,city,students_count
0,São Paulo,144993
1,Rio de Janeiro,63313
2,Brasília,42052
3,Manaus,31900
4,Fortaleza,30986
5,Salvador,29450
6,Belo Horizonte,26227
7,Curitiba,23798
8,Guarulhos,20252
9,Recife,19911


In [14]:
student_count_by_ethnicity_by_state_query = """
select
	state_abrv as state,
	student_ethnicity,
	count(1) as student_count
from
	public.censo c
where
	student_ethnicity != 'Não declarada'
group by
	state_abrv,
	student_ethnicity
order by
	state_abrv,
	student_ethnicity
"""

In [15]:
student_count_by_ethnicity_by_state = pd.read_sql_query(student_count_by_ethnicity_by_state_query, connection)
student_count_by_ethnicity_by_state

,state,student_ethnicity,student_count
0,AC,Amarela,1038
1,AC,Branca,25674
2,AC,Indígena,11248
3,AC,Parda,140169
4,AC,Preta,3159
...,...,...,...
130,TO,Amarela,2449
131,TO,Branca,72481
132,TO,Indígena,7769
133,TO,Parda,265730


In [79]:
import plotly.express as px
import plotly.graph_objects as go

for state in student_count_by_ethnicity_by_state["state"].drop_duplicates():
    student_count_by_ethnicity = student_count_by_ethnicity_by_state[student_count_by_ethnicity_by_state["state"] == state]
    
    fig = go.Figure(
        data=[go.Bar(x=student_count_by_ethnicity["student_ethnicity"], 
                     y=student_count_by_ethnicity["student_count"],
                     text=student_count_by_ethnicity["student_count"],
                     textposition='auto',
                    )
             ]
    )
    
    fig.update_layout(title_text=f'Distribuição de cor/raça no estado {state}', plot_bgcolor='rgb(255,255,255)')
    fig.update_yaxes(showticklabels=False)
    fig.show()